# Pendulum

In [1]:
import numpy as np
from scipy.integrate import solve_ivp, odeint
import deepdish as dd

def pendulum(t, thetas, g, l):
    theta, dot_theta = thetas # y0, y1
    dots = (dot_theta, -(g/l)*np.sin(theta))
    return  dots # y0_dot, y1_dot

def create_dataset(n_datapoints, timesteps, dt, angles_bound, length_bound, g):
    min_angle, max_angle = angles_bound
    min_len, max_len = length_bound
    
    tmin = 0.0
    tmax = timesteps*dt
    ts = np.linspace(tmin, tmax, timesteps)

    labels = [] # np.empty(n_datapoints) #
    cartesian = np.empty((n_datapoints, timesteps, 2)) # 2d of motion
    phase_space = np.empty((n_datapoints, timesteps, 2)) # 2 degrees of freedom

    for i in range(n_datapoints):
        initial_angle = (max_angle - min_angle) * np.random.random_sample() + min_angle 
        theta0 = np.radians(initial_angle) # initial anglee
        omega0 = 0.0 # initial velocity

        length = (max_len - min_len) * i/(n_datapoints-1) + min_len
        sol = solve_ivp(pendulum, [tmin, tmax], [theta0, omega0], t_eval = ts, args=(g,length))

        # save the x, y coordinated of the pendulum
        xy = np.zeros_like(sol.y)
        xy[0] = length*np.sin(sol.y[0])
        xy[1] = length*np.cos(sol.y[0])
        cartesian[i] = xy.T

        phase_space[i] = sol.y.T

        labels.append({'initial_angle': initial_angle, 
                       'initial_velocity': omega0, 
                       'gravity': g, 
                       'length': length})

        if i % 500 == 0:
            print(i, length, initial_angle)
    dataset = {'cartesian': cartesian, 'phase_space': phase_space, 'labels': labels}
    return dataset

# Train & In-Dist Test Set

In [5]:
dt = 0.01
timesteps = 2000
angles_bound = (10, 170)
g = 9.81
ang_str = '-'.join([str(a) for a in angles_bound])

In [21]:
length_bound = (1.0, 1.5)
n_datapoints = 10000
dataset_train = create_dataset(n_datapoints, timesteps, dt, angles_bound, length_bound, g)
len_str = '-'.join([f'{a:.2f}' for a in length_bound])
dd.io.save(f'../data/pendulum_n_{n_datapoints}_steps_{timesteps}_dt_{dt}_len_{len_str}_angle_{ang_str}_g_{g}.hd5', dataset_train)

0 1.2 90.0
500 1.2500200080032013 90.0
1000 1.3000400160064025 90.0
1500 1.3500600240096037 90.0
2000 1.400080032012805 90.0


# Test set 1

In [ ]:
length_bound = (0.90, 1.00)
n_datapoints = 1000
dataset_test1 = create_dataset(n_datapoints, timesteps, dt, angles_bound, length_bound, g)
len_str = '-'.join([f'{a:.2f}' for a in length_bound])
dd.io.save(f'../data/pendulum_n_{n_datapoints}_steps_{timesteps}_dt_{dt}_len_{len_str}_angle_{ang_str}_g_{g}.hd5', dataset_test1)

0 0.9 159.09978933331283
500 0.95005005005005 78.38651833296461


# Test set 2

In [7]:
length_bound = (1.50, 1.60)
n_datapoints = 1000
dataset_test2 = create_dataset(n_datapoints, timesteps, dt, angles_bound, length_bound, g)
len_str = '-'.join([f'{a:.2f}' for a in length_bound])
dd.io.save(f'../data/pendulum_n_{n_datapoints}_steps_{timesteps}_dt_{dt}_len_{len_str}_angle_{ang_str}_g_{g}.hd5', dataset_test2)

0 1.5 22.093442472969613
500 1.55005005005005 59.910835489055486
